In [125]:
#pandas libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import random

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingRegressor

from sklearn.metrics import log_loss
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import Normalizer

from scipy.stats import uniform

%run "../notebooks/cargar_df.ipynb"

### Cargo los data frames con los features

In [2]:
trainX = pd.read_csv('../files_csv/train_173_features_bin_encoding')
trainX

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD,Stage
0,0,1,1,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0.5200,0.58817,5.609362e+06,0
1,1,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0.5300,0.59948,5.130851e+04,1
2,2,0,0,1,1,0,1,0,1,0,...,1,0,0,0,1,0,0.4800,0.48000,8.386560e+04,1
3,3,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.5300,0.53000,7.421882e+06,0
4,4,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.5300,0.53000,1.335719e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,12799,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0.5150,0.58251,4.273404e+05,1
9787,12800,1,0,0,1,0,1,0,1,1,...,1,0,0,0,1,0,0.5300,0.53000,1.066625e+07,0
9788,12801,1,1,1,0,1,0,1,1,1,...,0,1,0,0,0,1,0.5200,0.58817,1.062819e+05,1
9789,12802,1,1,1,0,1,0,1,1,0,...,1,0,0,0,1,0,0.6375,0.63750,2.346797e+06,0


In [3]:
y = trainX['Stage']
y

0       0
1       1
2       1
3       0
4       0
       ..
9786    1
9787    0
9788    1
9789    0
9790    0
Name: Stage, Length: 9791, dtype: int64

In [4]:
ids = trainX.Opportunity_ID
ids

0           0
1           1
2           2
3           3
4           4
        ...  
9786    12799
9787    12800
9788    12801
9789    12802
9790    12803
Name: Opportunity_ID, Length: 9791, dtype: int64

In [5]:
trainX = trainX.drop(['Opportunity_ID', 'Stage'], axis=1)

In [6]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9791 entries, 0 to 9790
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 12.8 MB


In [7]:
testX = pd.read_csv('../files_csv/test_173_features_bin_encoding')
testX

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_1,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD
0,10689,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.306667,0.346870,137147.816349
1,10690,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.332000,0.375524,169716.349384
2,10691,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.425000,0.425000,21037.500000
3,10692,1,1,1,0,1,1,0,0,1,...,0,1,0,0,0,1,0,0.425000,0.425000,361517.750000
4,10693,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.390000,0.390000,5752.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,12364,1,1,1,1,0,1,0,1,1,...,0,1,0,0,0,1,0,0.300000,0.300000,73875.000000
1563,12365,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.410000,0.410000,45054.900000
1564,12366,1,1,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0,0.410000,0.410000,100122.000000
1565,12367,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.500000,0.500000,143220.000000


In [8]:
ids_test = testX.Opportunity_ID
ids_test

0       10689
1       10690
2       10691
3       10692
4       10693
        ...  
1562    12364
1563    12365
1564    12366
1565    12367
1566    12368
Name: Opportunity_ID, Length: 1567, dtype: int64

In [9]:
testX = testX.drop('Opportunity_ID', axis=1)

In [10]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 2.1 MB


In [83]:
# Del archivo 'prediccion_log_regression'
# Las columnas mas influyentes para predecir con Logistic Regression

train = trainX[['Bureaucratic_Code_2', 'Total_Amount_USD', \
                             'has_product_Category_B', 'bureaucratic_aprobada',\
                             'Bureaucratic_Code_3', 'Billing_Country_7', \
                             'has_product_Type', 'Last_Modified_By_2', \
                             'Account_Name_3', 'has_brand', 'Bureaucratic_Code_0_Approval_1',\
                             'Opportunity_Type_5', 'active_days', 'month']]

In [84]:
test = testX[['Bureaucratic_Code_2', 'Total_Amount_USD', \
                             'has_product_Category_B', 'bureaucratic_aprobada',\
                             'Bureaucratic_Code_3', 'Billing_Country_7', \
                             'has_product_Type', 'Last_Modified_By_2', \
                             'Account_Name_3', 'has_brand', 'Bureaucratic_Code_0_Approval_1',\
                             'Opportunity_Type_5', 'active_days', 'month']]

### Calculo el error

In [85]:
def calculateLogisticRegError(data, params):
    # Calculo el error
    X_train, X_test, y_train, y_test = \
        train_test_split(data, y, test_size=0.20, random_state=123)

    model = LogisticRegression(C = params['C'], \
                           class_weight = params['class_weight'], \
                           fit_intercept = params['fit_intercept'], \
                           max_iter = params['max_iter'], \
                           tol = params['tol'], \
                           warm_start = params['warm_start'])
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)
    ll = log_loss(y_test, preds, normalize=True)
    print("Log Loss: %f" % (ll))

In [86]:
params = {'C' : 1.2, \
          'class_weight' : None, \
          'fit_intercept' : True, \
          'max_iter' : 200, \
          'tol' : 0.0001, \
          'warm_start' : False}

calculateLogisticRegError(train, params)

Log Loss: 0.630771


In [87]:
train['Total_Amount_USD'] = train['Total_Amount_USD'].agg(lambda x: np.log(x+1))
test['Total_Amount_USD'] = test['Total_Amount_USD'].agg(lambda x: np.log(x+1))

/home/cecilia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/cecilia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [88]:
params = {'C' : 1.2, \
          'class_weight' : None, \
          'fit_intercept' : True, \
          'max_iter' : 200, \
          'tol' : 0.0001, \
          'warm_start' : False}

calculateLogisticRegError(train, params)

Log Loss: 0.487982


/home/cecilia/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [89]:
train['active_days'] = train['active_days'].agg(lambda x: np.log(x+1))
test['active_days'] = test['active_days'].agg(lambda x: np.log(x+1))

/home/cecilia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/cecilia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [90]:
params = {'C' : 1.2, \
          'class_weight' : None, \
          'fit_intercept' : True, \
          'max_iter' : 200, \
          'tol' : 0.0001, \
          'warm_start' : False}

calculateLogisticRegError(train, params)

Log Loss: 0.464919


In [ ]:
# Set the parameters by random search
tuned_parameters = [{'penalty': ['l1', 'l2'],
                     'tol': [0.0001, 0.0002, 0.00005],
                     #'C': [0.5, 0.8, 1.0, 1.2],
                     'C': uniform(loc=0, scale=4),
                     'fit_intercept': [True, False],
                     'class_weight': [None, 'balanced'],
                     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
                     'max_iter': [200, 300, 400], 
                     'multi_class': ['ovr'], 
                     'warm_start': [False, True]
    }]

score = 'log_loss'
print("Tuning hyper-parameters for %s with Logistic Regression\n" % score)

transformer = Normalizer().fit(train)
transformed_train = transformer.transform(train)
transformed_train

from scipy.stats import uniform
logistic = LogisticRegression()
clf = RandomizedSearchCV(logistic, tuned_parameters, \
                         random_state=123, scoring='neg_log_loss',\
                         n_iter=50)
search = clf.fit(transformed_train, y)

print("Best Params:")
print(search.best_params_ )
print()
print('Best Score:')
print(search.best_score_)

Resultado de la celda anterior

Best Params:
{'C': 1.6691642436133765, 'class_weight': None, 'fit_intercept': True, 'max_iter': 400, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.0002, 'warm_start': False}

Best Score:
-0.5167853376223811

In [99]:
params = {'C' : 1.6691642436133765, \
          'class_weight' : None, \
          'fit_intercept' : True, \
          'max_iter' : 400, \
          'multi_class' : 'ovr', \
          'penalty' : 'l1', \
          'solver' : 'liblinear', \
          'tol' : 0.0002, \
          'warm_start' : False}

calculateLogisticRegError(train, params)

Log Loss: 0.464816


In [101]:
model = LogisticRegression(C = 1.6691642436133765, \
                           class_weight = None, \
                           fit_intercept = True, \
                           max_iter = 400, \
                           multi_class = 'ovr',\
                           penalty = 'l1',\
                           solver = 'liblinear',\
                           tol = 0.0002, \
                           warm_start = False)


model.fit(train, y)
preds = model.predict_proba(test)

In [102]:
preds

array([[0.12769241, 0.87230759],
       [0.38955297, 0.61044703],
       [0.04337061, 0.95662939],
       ...,
       [0.19830301, 0.80169699],
       [0.03475183, 0.96524817],
       [0.41150612, 0.58849388]])

In [103]:
preds.shape

(1567, 2)

In [104]:
pd.DataFrame(preds)[1]

0       0.872308
1       0.610447
2       0.956629
3       0.641254
4       0.971554
          ...   
1562    0.932943
1563    0.882274
1564    0.801697
1565    0.965248
1566    0.588494
Name: 1, Length: 1567, dtype: float64

In [105]:
prediccion = pd.DataFrame({'Opportunity_ID':ids_test, 'Target':pd.DataFrame(preds)[1]})
prediccion

,Opportunity_ID,Target
0,10689,0.872308
1,10690,0.610447
2,10691,0.956629
3,10692,0.641254
4,10693,0.971554
...,...,...
1562,12364,0.932943
1563,12365,0.882274
1564,12366,0.801697
1565,12367,0.965248


In [106]:
prediccion.to_csv('prediccion_logistic_regression_4', index=False)